<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=bf5366a0eb033e06ff7ac4744479bb34abbb872333df21f892f40df80fab333f
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-24 11:47:47
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -53.35 K (-0.37%)
Current PnL: -23.11 L (-15.15%)
CY Booked + Current PnL: -9.22 L (-6.04%)
-------------------
Total profit:  1.54 L
Total loss:  -24.66 L
-------------------
Total Booked + Current PnL: 17.76 L (14.13%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.74%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.53 L (62.05%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.25%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.20,-14.06,16.47,0.09,14264.0,-14173.0,86604.0,101.07,50.0,M-SC,4.26,253.0,-0.99,0.61,13.78,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.06,-8.93,9.83,0.02,15627.0,-15591.0,158968.0,142.19,71.0,M-SC,5.45,234.0,-1.00,1.11,38.27,OX40N,NTT,PAINTS
35,ICICIGI,2252.93,0.32,7.26,12.06,20.19,22549.0,12656.0,186976.0,-16.02,52.0,X-MC,7.66,68.0,0.56,1.31,23.47,X40,ATH,INSURANCE
43,ITC,452.00,-0.69,-1.03,11.59,10.44,22956.0,-2069.0,198069.0,-40.74,45.0,X-LC,0.77,5.0,-0.09,1.39,4.47,X40,NTT,FMCG
53,NESTLEIND,1377.00,-0.84,11.96,8.43,21.40,25052.0,31754.0,297180.0,3.91,55.0,X-LC,7.90,12.0,1.27,2.08,19.40,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.00,1.95,-20.96,84.18,45.57,131157.0,-41327.0,155805.0,3370.27,43.0,M-SC,4.00,244.0,-0.32,1.09,10.46,XY24,NTT,AUTO
26,GILLETTE,11206.78,1.61,-0.63,33.26,32.42,83915.0,-1596.0,252300.0,-18.36,36.0,X-SC,6.64,85.0,-0.02,1.77,14.38,X40,ATH,FMCG
68,SONACOMS,806.63,1.42,-11.72,58.07,39.54,51858.0,-11857.0,89302.0,-30.90,69.0,M-SC,6.86,220.0,-0.23,0.63,25.93,AR,ATH,AUTO
85,WIPRO,420.00,1.41,1.69,69.40,72.27,119584.0,2863.0,172311.0,-12.89,58.0,M-LC,4.60,99.0,0.02,1.21,7.97,XR,NTT,IT
32,HCLTECH,1922.01,1.37,5.78,17.91,24.73,45833.0,13986.0,255910.0,10.64,64.0,X-LC,4.64,7.0,0.31,1.79,21.90,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VOLTAS,1530.00,-2.82,6.12,12.79,19.69,26024.0,11733.0,203475.0,-4.26,46.0,X-MC,3.51,78.0,0.45,1.43,13.33,XY25,NTT,AC
16,CERA,9475.00,-2.79,-29.02,74.54,23.89,93068.0,-51047.0,124856.0,-31.50,25.0,H-SC,3.99,157.0,-0.55,0.88,7.83,OX40N,NTT,CERAMICS
27,GLAXO,3466.20,-2.67,-2.41,42.58,39.15,82813.0,-4796.0,194488.0,-21.65,28.0,X-MC,9.08,60.0,-0.06,1.36,25.90,X40,ATH,PHARMA
81,VALIANTORG,838.00,-2.49,-45.57,219.97,74.15,195874.0,-74559.0,89046.0,-260.32,26.0,H-SC,14.55,149.0,-0.38,0.62,14.14,XR,NTT,CHEMICALS
72,TANLA,1943.92,-2.35,-36.26,244.67,119.69,422261.0,-98177.0,172584.0,-35.52,27.0,H-SC,12.03,156.0,-0.23,1.21,36.34,AR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.91,-0.28,107.31,106.72,131976.0,-350.0,122986.0,-50.63,33.0,H-SC,4.09,139.0,-0.0,0.86,23.03,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.94,-4.89,118.60,107.91,166548.0,-7216.0,140428.0,-22.58,33.0,M-SC,10.35,216.0,-0.04,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.06,-8.93,9.83,0.02,15627.0,-15591.0,158968.0,142.19,71.0,M-SC,5.45,234.0,-1.00,1.11,38.27,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.20,-14.06,16.47,0.09,14264.0,-14173.0,86604.0,101.07,50.0,M-SC,4.26,253.0,-0.99,0.61,13.78,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.51,-22.70,46.79,13.47,97537.0,-61209.0,208458.0,-68.31,26.0,H-SC,1.52,158.0,-0.63,1.46,5.99,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,0.54,-22.91,71.27,32.03,72432.0,-30204.0,101631.0,-16.54,49.0,H-SC,6.63,148.0,-0.42,0.71,13.84,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,1.41,1.69,69.4,72.27,119584.0,2863.0,172311.0,-12.89,58.0,M-LC,4.6,99.0,0.02,1.21,7.97,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.41,1.69,69.40,72.27,119584.0,2863.0,172311.0,-12.89,58.0,M-LC,4.60,99.0,0.02,1.21,7.97,XR,NTT,IT
38,INDIAMART,4810.62,-0.91,-0.28,107.31,106.72,131976.0,-350.0,122986.0,-50.63,33.0,H-SC,4.09,139.0,-0.00,0.86,23.03,AR,ATH,MISC
25,FINCABLES,1641.55,-0.94,-4.89,118.60,107.91,166548.0,-7216.0,140428.0,-22.58,33.0,M-SC,10.35,216.0,-0.04,0.98,0.00,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,0.04,-2.49,39.09,35.63,79884.0,-5215.0,204359.0,-16.30,32.0,H-MC,4.51,119.0,-0.07,1.43,13.77,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,-0.45,-5.83,93.19,81.92,100738.0,-6698.0,108100.0,-44.81,31.0,M-SC,5.30,236.0,-0.07,0.76,27.97,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-1.39,-15.36,67.97,42.18,159027.0,-42444.0,233966.0,-24.30,22.0,X-LC,0.43,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
46,JSWINFRA,342.00,-1.20,-10.16,28.26,15.23,50865.0,-20358.0,179989.0,-23.19,23.0,X-MC,6.52,66.0,-0.40,1.26,18.81,X40N,NTT,REALTY
11,BATAINDIA,2096.00,0.52,-35.13,108.43,35.20,90501.0,-45205.0,83465.0,9.46,25.0,X-SC,12.61,92.0,-0.50,0.59,0.52,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,0.56,-15.70,70.01,43.32,96124.0,-25572.0,137300.0,-27.57,27.0,X-SC,2.02,90.0,-0.27,0.96,5.42,X40N,ATH,ELECTRICAL
60,RELAXO,1176.00,-1.05,-47.66,190.98,52.31,145107.0,-69180.0,75980.0,-44.25,28.0,X-SC,3.79,91.0,-0.48,0.53,1.70,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-1.39,-15.36,67.97,42.18,159027.0,-42444.0,233966.0,-24.30,22.0,X-LC,0.43,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
43,ITC,452.0,-0.69,-1.03,11.59,10.44,22956.0,-2069.0,198069.0,-40.74,45.0,X-LC,0.77,5.0,-0.09,1.39,4.47,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.02,1.29,29.30,30.97,52454.0,2285.0,179023.0,-22.52,49.0,X-MC,1.54,75.0,0.04,1.25,14.02,X40,ATH,INSURANCE
3,ACC,3906.0,-0.38,-23.38,114.25,64.15,208289.0,-55641.0,182310.0,-55.00,41.0,X-SC,1.90,82.0,-0.27,1.28,2.27,XY24,BTT,CEMENT
66,SIEMENS,4671.5,-0.11,-14.83,47.36,25.51,75068.0,-27590.0,158505.0,-2.66,53.0,H-LC,1.93,49.0,-0.37,1.11,16.81,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.05,-29.34,102.15,42.84,46923.0,-19071.0,45935.0,-53.72,36.0,X-SC,37.28,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
76,TMPV,600.00,-1.39,-15.36,67.97,42.18,159027.0,-42444.0,233966.0,-24.30,22.0,X-LC,0.43,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.45,-13.30,71.23,48.45,123826.0,-26666.0,173840.0,-19.44,34.0,X-MC,7.45,64.0,-0.22,1.22,0.45,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,0.52,-35.13,108.43,35.20,90501.0,-45205.0,83465.0,9.46,25.0,X-SC,12.61,92.0,-0.50,0.59,0.52,X40,NTT,FOOTWEAR
54,PAGEIND,51605.07,0.08,-4.93,32.61,26.07,50761.0,-8080.0,155660.0,-29.94,36.0,X-MC,8.86,55.0,-0.16,1.09,0.91,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.05,-29.34,102.15,42.84,46923.0,-19071.0,45935.0,-53.72,36.0,X-SC,37.28,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-1.05,-47.66,190.98,52.31,145107.0,-69180.0,75980.0,-44.25,28.0,X-SC,3.79,91.0,-0.48,0.53,1.70,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.52,-35.13,108.43,35.20,90501.0,-45205.0,83465.0,9.46,25.0,X-SC,12.61,92.0,-0.50,0.59,0.52,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,0.02,2.47,21.09,24.08,27435.0,3133.0,130083.0,-5.81,44.0,X-SC,4.75,89.0,0.11,0.91,22.57,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-1.05,-4.90,27.85,21.58,37116.0,-6867.0,133272.0,-52.09,33.0,X-MC,7.02,56.0,-0.19,0.93,17.99,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.15,-14.52,39.55,19.28,116955.0,-50241.0,295715.0,-25.57,62.0,X-LC,7.64,1.0,-0.43,2.07,9.32,X40,ATH,IT
41,INFY,2275.00,1.13,6.36,45.61,54.87,151786.0,19903.0,332791.0,-17.09,63.0,X-LC,3.15,2.0,0.13,2.33,15.17,X40,BTT,IT
76,TMPV,600.00,-1.39,-15.36,67.97,42.18,159027.0,-42444.0,233966.0,-24.30,22.0,X-LC,0.43,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
82,VBL,671.64,-0.47,-10.00,50.74,35.67,144234.0,-31581.0,284261.0,-20.77,40.0,X-LC,6.06,4.0,-0.22,1.99,2.47,X40N,ATH,FMCG
43,ITC,452.00,-0.69,-1.03,11.59,10.44,22956.0,-2069.0,198069.0,-40.74,45.0,X-LC,0.77,5.0,-0.09,1.39,4.47,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-2.25,-14.72,113.03,81.67,90426.0,-13808.0,80002.0,7188.89,46.0,L-SC,18.48,271.0,-0.15,0.56,54.93,XR,NTT,CERAMICS
68,SONACOMS,806.63,1.42,-11.72,58.07,39.54,51858.0,-11857.0,89302.0,-30.90,69.0,M-SC,6.86,220.0,-0.23,0.63,25.93,AR,ATH,AUTO
51,MASFIN,398.61,-2.00,-4.87,28.29,22.05,26369.0,-4770.0,93210.0,-17.80,51.0,H-SC,7.22,164.0,-0.18,0.65,35.49,XR,ATH,FINANCE
70,SURYODAY,216.00,-0.82,-20.32,54.41,23.03,77637.0,-36383.0,142688.0,55.70,47.0,H-SC,11.07,167.0,-0.47,1.00,41.57,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.06,-8.93,9.83,0.02,15627.0,-15591.0,158968.0,142.19,71.0,M-SC,5.45,234.0,-1.00,1.11,38.27,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.06,-8.93,9.83,0.02,15627.0,-15591.0,158968.0,142.19,71.0,M-SC,5.45,234.0,-1.00,1.11,38.27,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.26,0.14,0.69,20.12,20.95,51003.0,1725.0,253493.0,-0.35,76.0,X-LC,12.74,31.0,0.03,1.78,36.57,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.44,-37.22,111.69,32.89,54133.0,-28740.0,48467.0,-698.49,65.0,L-MC,5.62,259.0,-0.53,0.34,33.47,XR,NTT,BANKS
50,LTIM,7201.88,1.21,9.64,20.08,31.66,54194.0,23737.0,269888.0,5.43,65.0,H-LC,12.55,43.0,0.44,1.89,49.75,X200,ATH,IT
0,5PAISA,593.00,-2.00,-36.03,76.49,12.90,106915.0,-78720.0,139776.0,105.53,56.0,H-SC,14.56,161.0,-0.74,0.98,16.22,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.81
1,25,44.69
2,50,75.47


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.54
MC    29.26
LC    25.19
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.89
X40      21.25
X40N     11.84
XR        9.63
XY25      9.08
AR        8.97
OX40N     7.67
X200      1.89
SR        0.99
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.91
X-MC    22.85
X-LC    20.01
M-SC    12.06
X-SC     8.18
H-MC     4.70
H-LC     3.00
L-SC     1.39
M-MC     1.37
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.80,-5.02,38.98
IT,13.85,-14.26,73.24
FINANCE,9.43,-15.15,63.81
MISC,7.41,-22.72,72.99
PAINTS,6.00,-7.82,23.83
ELECTRICAL,5.89,-11.65,52.54
INSURANCE,4.41,-0.34,35.70
PHARMA,3.83,-2.22,34.93
AUTO,3.36,-19.96,71.40


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3056951.0,21
AR,1291927.0,10
XR,1288969.0,13
X40,1006704.0,14
X40N,757351.0,9
OX40N,696855.0,10
XY25,346669.0,6
SR,279553.0,2
MH,73402.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3566756.0,25
M-SC,1338982.0,15
X-MC,1222986.0,16
X-LC,907006.0,11
X-SC,752217.0,8
H-MC,408046.0,3
L-SC,262771.0,3
H-LC,129262.0,2
M-LC,119584.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1221551.0      6
           AR         876647.0      5
           XR         791450.0      7
M-SC       XY24       783150.0      6
X-MC       X40        443755.0      7
X-LC       X40        388533.0      5
X-MC       XY24       348880.0      3
H-SC       OX40N      324153.0      4
X-SC       X40N       295154.0      3
M-SC       OX40N      284238.0      5
X-SC       XY24       282647.0      3
H-SC       SR         279553.0      2
X-MC       X40N       248368.0      4
X-LC       XY24       230790.0      2
H-MC       AR         218113.0      2
X-LC       X40N       213829.0      2
H-MC       XY24       189933.0      1
X-MC       XY25       181983.0      2
X-SC       X40        174416.0      2
L-SC       XR         174307.0      2
M-SC       XR         149495.0      2
           AR         122099.0      2
M-LC       XR         119584.0      1
L-SC       OX40N       88464.0      1
H-LC       AR          75068.0      1
X-LC       XY25        73854.0      2
H-SC       MH          73402.0      1
H-LC       X200        54194.0      1
L-MC       XR          54133.0      1
M-MC       XY25        49966.0      1
L-LC       XY25        40866.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
